In [6]:
import xmltodict
import json
from glob import glob
import sys, os
from tqdm import tqdm
import pandas as pd


In [ ]:
## You need to download dataset
## Check https://alt.qcri.org/semeval2016/task5/index.php?id=data-and-tools

In [2]:

# filenames = glob('raw_dataset/*.xml')

# for fname in filenames:
#     o = xmltodict.parse(open(fname, 'rt').read())
#     fn = fname.split('/')[-1].split('.')[0]
#     save_name = os.path.join('raw_dataset', f"{fn}.json")
#     print(save_name)
#     with open(save_name, 'wt') as f:
#         json.dump(o, f, indent=4, sort_keys=True)

In [7]:
jsonfilenames = glob('raw_dataset/*.json')

In [8]:


# def remove_null(line):
#     res=[]
#     line=line.split(',')
#     for l in line:
#         if not 'NULL' in l:
#             res.append(l)
#     return ' , '.join(res)

# def sub_null(line):
#     res=[]
#     line=line.split(',')
#     for l in line:
#         if not 'NULL' in l:
#             res.append(l)
#         else:
#             l = l.replace('NULL', '')
#             l = l.replace('  ', ' ')
#             res.append(l)
#     return ' , '.join(res)



def remove_null(line):
    res=[]
    line=line.split("<&>") # ","
    for l in line:
        if not 'NULL' in l:
            res.append(l.strip())
    return ' <&> '.join(res)

# def sub_null(line):
# #     res=[]
# #     line=line.split(',')
# #     for l in line:
# #         l = l.replace('NULL', 'None')
# #         l = l.replace('  ', ' ')
# #         res.append(l)
    
#     return line.replace('NULL','None')

def rm_dup(line):
#     line = line.
    return ' <&> '.join(list(dict.fromkeys(line.split(' <&> '))))
    

In [9]:
def json2tsv(filename):
    with open(filename,'r') as f:
        data=json.load(f)

    f.close()
    sentences = []
    polarity = []
    term = []
    labels = []
    category = []
    labels2 = []
    bio_labels = []
    for d in data['Reviews']['Review']:
        infos = d['sentences']['sentence']
        if type(infos) == dict:
            infos = [infos]
        for review in infos:
            tmp_pol = []
            tmp_term = []
            tmp_cat = []
#             print(review)
            if 'Opinions' not in review:
                continue
            if review['Opinions']:
                sentences.append(review['text'])
                opinion = review['Opinions']['Opinion']
                if type(opinion) == dict:
                    tmp_pol.append(opinion['@polarity'])
                    tmp_term.append(opinion['@target'])
                    tmp_cat.append(opinion['@category'])
                    
                    lb = opinion['@target'] + ' ' + opinion['@polarity']
                    lb2 = opinion['@category'].split('#')[0].lower() + ' ' + opinion['@target'] + ' ' + opinion['@polarity']
                    s, e = int(opinion['@from']) , int(opinion['@to'])
#                     if review['text'] == 'Servicio esmerado.':
#                         print(s, e)
                    lb_bio=['O' for _ in range(len(review['text'].split(' ')))]
                    if opinion['@target'] != 'NULL':

                        if (s and e) or (s==0):
                            tmp_bio=review['text'].replace(review['text'][s:e], '<TERM>'+review['text'][s:e]+'<TERM>')
                            lst=[i for i, word in enumerate(tmp_bio.split()) if '<TERM>' in word]
                            if len(lst)==1:
                                lb_bio[lst[0]] = 'S-{}'.format(opinion['@polarity'])
                            else:
                                lst = [i for i in range(lst[0],lst[1]+1)]
                                lb_bio[lst[0]] = 'B-{}'.format(opinion['@polarity'])

                                for idx in lst[1:-1]:
                                        lb_bio[idx] = 'I-{}'.format(opinion['@polarity'])
                                lb_bio[lst[-1]] = 'E-{}'.format(opinion['@polarity'])

                else:
                    lb = ''
                    lb2 = ''
                    lb_bio = ['O' for _ in range(len(review['text'].split(' ')))]
                    for opinion in review['Opinions']['Opinion']:
        #             print(opinion)
                        tmp_pol.append(opinion['@polarity'])
                        tmp_term.append(opinion['@target'])
                        tmp_cat.append(opinion['@category'])
                    
                        l = opinion['@target'] + ' ' + opinion['@polarity']
                        l2 = opinion['@category'].split('#')[0].lower() + ' ' + opinion['@target'] + ' ' + opinion['@polarity']


                        lb += l
                        lb += ' <&> '

                        lb2 += l2
                        lb2 += ' <&> '
                    
                        s, e = int(opinion['@from']) , int(opinion['@to'])
                        if opinion['@target'] != 'NULL':
                            if (s and e) or (s==0):
                                tmp_bio=review['text'].replace(review['text'][s:e], '<TERM>'+review['text'][s:e]+'<TERM>')
                                lst=[i for i, word in enumerate(tmp_bio.split()) if '<TERM>' in word]
                                if len(lst)==1:
                                    lb_bio[lst[0]] = 'S-{}'.format(opinion['@polarity'])
                                else:
                                    lst = [i for i in range(lst[0],lst[1]+1)]
                                    lb_bio[lst[0]] = 'B-{}'.format(opinion['@polarity'])

                                    for idx in lst[1:-1]:
                                            lb_bio[idx] = 'I-{}'.format(opinion['@polarity'])

                                    lb_bio[lst[-1]] = 'E-{}'.format(opinion['@polarity'])
                if lb.strip()[-3:] == '<&>':
                    lb = lb[:-4]
                else:
                    lb = lb.strip()
                if lb2.strip()[-3:] == '<&>':
                    lb2 = lb2[:-4]
                else:
                    lb2 = lb2.strip()
                polarity.append(' <&> '.join(tmp_pol))
                term.append(' <&> '.join(tmp_term))
                labels.append(lb)
                labels2.append(lb2)
                
                
                    
                category.append(' , '.join(tmp_cat))
                bio_labels.append(' '.join(lb_bio))

    df=pd.DataFrame()
    df['review'] = sentences
    df['labels'] = labels
    df['labels'] = df['labels'].map(lambda x: rm_dup(x))
    df['category'] = category
#     df['labels_acsd'] = labels2
    
    df['labels_bieos'] = bio_labels

    df['labels_rm'] = df['labels'].map(lambda x: remove_null(x))
    df['labels_rm'] = df['labels_rm'].map(lambda x: rm_dup(x))
    
#     df['labels_acsd_rm'] = df['labels_acsd'].map(lambda x: remove_null(x))


#     df['labels_sub'] = df['labels'].map(lambda x: sub_null(x))
#     df['labels_acsd_sub'] = df['labels_acsd'].map(lambda x: sub_null(x))     
    
    

    for i in range(len(df)):
        tmp=df['review'][i].split(' ')
        tmp_lb = df['labels_bieos'][i].split(' ')
        res = []
        res_lb = []
        for t, b in zip(tmp, tmp_lb):
            if t != '':
                res.append(t)
                res_lb.append(b)
        assert len(res) == len(res_lb)
        df['review'][i] = ' '.join(res)
        df['labels_bieos'][i] = ' '.join(res_lb)

    
    
    
    lang, dt = filename.split('/')[1].split('_')[2], filename.split('/')[1].split('_')[-1].split('.json')[0]
    
    if lang=='du': lang = 'nl'
    new_name =  'datasets/res16_' + lang + '_' + dt + '_bieos.tsv'
    df.to_csv(new_name, index=False, sep='\t')
#     return df
    print('>> from {} to {}'.format(filename, new_name))
    

In [10]:
for fn in jsonfilenames:
    json2tsv(fn)

>> from raw_dataset/ABSA16_res_en_train.json to datasets/res16_en_train_bieos.tsv
>> from raw_dataset/ABSA16_res_du_train.json to datasets/res16_nl_train_bieos.tsv
>> from raw_dataset/ABSA16_res_fr_test.json to datasets/res16_fr_test_bieos.tsv
>> from raw_dataset/ABSA16_res_es_test.json to datasets/res16_es_test_bieos.tsv
>> from raw_dataset/ABSA16_res_ru_train.json to datasets/res16_ru_train_bieos.tsv
>> from raw_dataset/ABSA16_res_tu_train.json to datasets/res16_tu_train_bieos.tsv
>> from raw_dataset/ABSA16_res_en_test.json to datasets/res16_en_test_bieos.tsv
>> from raw_dataset/ABSA16_res_fr_train.json to datasets/res16_fr_train_bieos.tsv
>> from raw_dataset/ABSA16_res_ru_test.json to datasets/res16_ru_test_bieos.tsv
>> from raw_dataset/ABSA16_res_tu_test.json to datasets/res16_tu_test_bieos.tsv
>> from raw_dataset/ABSA16_res_du_test.json to datasets/res16_nl_test_bieos.tsv
>> from raw_dataset/ABSA16_res_es_train.json to datasets/res16_es_train_bieos.tsv
